In [1]:
import time
import re
import pandas as pd
from tqdm import tqdm
from numba import jit

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
jit(nopython = True, parallel = True)

def find_good_info(name):
    
    data = {}
    
    driver.get('https://www.tmall.com/?spm=a220m.1000858.a2226n0.1.698c72d17SoGIg')
    driver.find_element_by_xpath('//*[@id="mq"]').clear()
    driver.find_element_by_xpath('//*[@id="mq"]').send_keys(name)
    driver.find_element_by_xpath('//*[@id="mq"]').submit()
    
    # explicitly wait until the page has finished loading 
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'product')))
    
    # product not found
    try:
        if driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/p[1]').text[0] == "喵":
            data = {}
            return data
    except:
        pass
    
    # locate official shop 
    shop_lst = driver.find_elements_by_class_name('productShop-name')
    for shop in shop_lst:
        if "旗舰" in shop.text:
            idx = shop_lst.index(shop)
            break
            
    # locate product url
    prodimg_lst = driver.find_elements_by_class_name('productImg')
    prod_img = prodimg_lst[idx]
    
    # record product url
    url = prod_img.get_attribute('href')
    
    # click the product
    prod_table = driver.find_elements_by_class_name('product')
    prod_table[idx].click()
    time.sleep(3)
    
    # login window jumps out
    handles = driver.window_handles
    driver.switch_to.window(handles[1])
    
    # close login window
    try:
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH,'/html/body/div[9]/div[2]/div')))
        driver.find_element_by_xpath('/html/body/div[9]/div[2]/div').click()
    except:
        pass
   
    # product information scrapy 
    tmallname = driver.find_element_by_class_name('tb-detail-hd').find_element_by_xpath('h1').text.strip('')
    price = driver.find_element_by_class_name('tm-price').text
    
    popularity = driver.find_element_by_xpath('//*[@id="J_CollectCount"]').text
    popularity = re.findall('[0-9]+', popularity)[0]
    
    info = driver.find_element_by_xpath('//*[@id="J_AttrUL"]').text
    brand = re.findall('品牌.\s(.*)', info)[0]
    
    # explicitly wait until the reviews have finished loading 
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'J_ReviewsCount')))
    driver.find_element_by_class_name('J_ReviewsCount').click()
    
    # explicitly wait until the rate has finished loading 
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'rate-score')))
    
    score = driver.find_element_by_class_name('rate-score').text
    score = re.findall('[0-9]\.*[0-9]*', score)[0]
    
    posi_elements = driver.find_elements_by_class_name("tag-posi")
    neg_elements = driver.find_elements_by_class_name("tag-neg")
    posi = ''
    neg = ''
    
    for i in posi_elements:
        posi = posi + ', ' + i.text
    posi = posi.strip(',')
    posi = posi.strip()
    for i in neg_elements:
        neg = neg + ', ' + i.text
    neg = neg.strip(',')
    neg = neg.strip()
    
    # close the product info page, back to search page
    if len(handles) > 1:
        driver.close()
    driver.switch_to.window(handles[0])
    
    # record data
    data = {'url':url, 'bid':bid, 'name':name, 'tmallname':tmallname, 'brand':brand, 
            'price':price, 'popularity':popularity, 'score':score, 'posi':posi, 'neg':neg}
    
    return data  

<function numba.decorators._jit.<locals>.wrapper(func)>

In [4]:
jit(nopython = True, parallel = True)

def try_more_times(item, count):
    
    try:
        data = find_good_info(product)
        return data
    
    except:
        handles = driver.window_handles
        
        if len(handles) > 1:
            driver.close()
            
        driver.switch_to.window(handles[0])
        count += 1
        
        if count >= 2:
            data = {}
            return data
        
        data = try_more_times(item, count)
        return data
    
    driver.close()

<function numba.decorators._jit.<locals>.wrapper(func)>

In [5]:
def mkdir(path):
    import os
    
#     path = path.strip()
#     path = path.rstrip("\\")
    isExists = os.path.exists(path)
 
    if not isExists:
        os.makedirs(path) 
        print(path + '\nFolder Create Successfully!')
        return True
    else:
        print(path + '\nFolder Already Exist!')
        return False

In [9]:
# input the index of final_uid_list
m = 11
n = m + 1

driver = webdriver.Firefox()

# web scarpy with T-Mall
for j in tqdm(range(m,n)):

    uid_list = list(pd.read_csv('final_uid_list/final_uid_list' + str(j) + '.csv', header = None)[1])
    
    # create a new folder under the parent folder of product_info
    mkpath = "product_info/" + str(j)
    mkdir(mkpath)

    for uid in tqdm(uid_list):
        try:
            trypath = 'product_info/' + str(j) + '/' + str(uid) + '_prodinfo.csv' 
            test = pd.read_csv(trypath)
            print('Product Information: ' + str(uid) + "\nAlready Exist!")
            
        except:
            try:
                prodinfo = []

                inpath = 'product_name/'+ str(j) + '/' + str(uid) + '_prodname.csv'
                prodnames = pd.read_csv(inpath, keep_default_na = False)

                for i in range(len(prodnames)):
                    for product in prodnames['产品'][i].split(', '):
                        count = 0
                        bid = prodnames['bid'][i]
                        tmallinfo = try_more_times(product, count)    

                        if len(tmallinfo) != 0:
                            prodinfo.append(tmallinfo)

                prodinfo = pd.DataFrame(prodinfo)

                if len(prodinfo) > 1:
                    prodinfo = prodinfo[['url', 'bid', 'name', 'tmallname', 'brand', 'price', 'popularity', 'score', 'posi', 'neg']]

                outpath = 'product_info/' + str(j) + '/' + str(uid) + '_prodinfo.csv' 
                prodinfo.to_csv(outpath, encoding = 'utf_8_sig')
            
            except Exception as e:
                print('Error: ', e)

  0%|          | 0/1 [00:00<?, ?it/s]

product_info/11
Folder Already Exist!


False


 23%|██▎       | 42/180 [00:00<00:00, 410.41it/s]

Product Information: 5464559534
Already Exist!
Product Information: 5466270837
Already Exist!
Product Information: 5467370887
Already Exist!
Product Information: 5467419946
Already Exist!
Product Information: 5474677446
Already Exist!
Product Information: 5476439893
Already Exist!
Product Information: 5478158468
Already Exist!
Product Information: 5482019409
Already Exist!
Product Information: 5484979713
Already Exist!
Product Information: 5492607158
Already Exist!
Product Information: 5494054350
Already Exist!
Product Information: 5494694476
Already Exist!
Product Information: 5496465159
Already Exist!
Product Information: 5498194953
Already Exist!
Product Information: 5500589234
Already Exist!
Product Information: 5500902147
Already Exist!
Product Information: 5501780065
Already Exist!
Product Information: 5502007055
Already Exist!
Product Information: 5507169462
Already Exist!
Product Information: 5519216861
Already Exist!
Product Information: 5519348187
Already Exist!
Product Infor


 95%|█████████▌| 171/180 [02:52<00:04,  1.85it/s]

Product Information: 5665162965
Already Exist!
Product Information: 5666687855
Already Exist!
Product Information: 5672538124
Already Exist!
Product Information: 5673985460
Already Exist!
Product Information: 5675789231
Already Exist!
Product Information: 5682369421
Already Exist!
Product Information: 5682720321
Already Exist!
Product Information: 5685649919
Already Exist!
Product Information: 5687295576
Already Exist!
Product Information: 5690439943
Already Exist!
Product Information: 5692001392
Already Exist!
Product Information: 5693657267
Already Exist!
Product Information: 5695934642
Already Exist!
Product Information: 5699464819
Already Exist!
Product Information: 5699800502
Already Exist!
Product Information: 5699800503
Already Exist!
Product Information: 5699988659
Already Exist!
Product Information: 5700403336
Already Exist!
Product Information: 5706006841
Already Exist!
Product Information: 5708869146
Already Exist!
Product Information: 5709992753
Already Exist!
Product Infor

100%|██████████| 1/1 [02:52<00:00, 172.31s/it]

Product Information: 5861396948
Already Exist!
Product Information: 5862241056
Already Exist!
Product Information: 5864576736
Already Exist!
Product Information: 5872499358
Already Exist!
Product Information: 5873824596
Already Exist!
Product Information: 5876462911
Already Exist!
Product Information: 5876770971
Already Exist!
Product Information: 5878732434
Already Exist!
Product Information: 5880911421
Already Exist!


In [ ]:
# input the index of final_uid_list
m = 0 
n = m + 1

# check the number of brands
scrapy_prods = 0
scrapy_brands = []
scrapy_uid = []
unscrapy_uid = []

lstpath = 'final_uid_list/' + 'final_uid_list' + str(m) + '.csv'
uid_list = pd.read_csv(lstpath, header = None)[1].tolist()

for uid in uid_list:
    try:
        csvpath = 'product_info/' + str(m) + '/' + str(uid) +'_prodinfo.csv' 
        prodinfo = pd.read_csv(csvpath, index_col = 0)
        
        if uid not in scrapy_uid:
            scrapy_uid.append(uid)
            
        scrapy_prods += len(prodinfo)
        
        if len(prodinfo) != 0:
            for i in prodinfo['brand']:
                if i not in scrapy_brands:
                    scrapy_brands.append(i)
    except:
        if uid not in unscrapy_uid:
            unscrapy_uid.append(uid)
                
print("No. of Scrapy Products: " + str(scrapy_prods))
print("\nNo. of Scrapy Brands: " + str(len(scrapy_brands)))
print("\nNo. of Scrapy UID: " + str(len(scrapy_uid)))
print("\nNo. of Unscrapy UID: " + str(len(unscrapy_uid)))
print("\n:)")